In [41]:
import pandas as pd 
from datetime import datetime, timedelta, date
import numpy as np

In [42]:
def porcentaje(asignadas, realizados):
    return (sum(realizados)/sum(asignadas))*100

In [43]:
tickets = pd.read_excel(r"C:\Users\centro.control\Downloads\Tickets de Recursos Humanos (CTS) (respuestas) (46).xlsx")
rutinas = pd.read_excel(r"C:\Users\centro.control\Downloads\CUMPLIMIENTO DE RUTINAS (84).xlsx", header=1)
vacantes = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Actualizacion de informacion\Vacantes 2025.xlsx")
ub = pd.read_excel(r"C:\Users\centro.control\Downloads\activos-universida-bennetts23-07-2025_8_53am.xlsx")


c:\Users\centro.control\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell Z349 is marked as a date but the serial value 6948701.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


# TICKETS

In [44]:
tickets['Fecha compromiso de cobertura']= pd.to_datetime(tickets['Fecha compromiso de cobertura'], errors='coerce').dt.date
tickets['Marca temporal']= pd.to_datetime(tickets['Marca temporal'], errors='coerce').dt.date
tickets['Fecha de cierre']= pd.to_datetime(tickets['Fecha de cierre'], errors='coerce').dt.date
hoy = (datetime.today()).date()
ayer = (datetime.today()-timedelta(days=1)).date()
antiayer = (datetime.today()-timedelta(days=2)).date()
mañana = (datetime.today()+timedelta(days=1)).date()
pasado_mañ = (datetime.today()+timedelta(days=2)).date()
trasmañana = (datetime.today()+timedelta(days=3)).date()
ticket_abiertos_proceden = tickets[
    (tickets['Estatus'] == 'Abierto') &
    (tickets['Procede'] == 'Procede') &
    (tickets['Marca temporal'] > datetime.strptime('2025-01-01', '%Y-%m-%d').date())
]

ticket_abiertos = tickets[tickets['Estatus']== 'Abierto']  





ticket_cerrados = sum(tickets['Fecha de cierre'] == hoy)+ sum(tickets['Fecha de cierre'] == ayer)  
# + sum(tickets['Marca temporal'] == antiayer)
ticket_nuevos = sum(tickets['Marca temporal'] == hoy)+ sum(tickets['Marca temporal'] == ayer)
ticket_xvencer = sum(ticket_abiertos['Fecha compromiso de cobertura'] == mañana)
# + sum(ticket_abiertos['Fecha compromiso de cobertura'] == pasado_mañ) + sum(ticket_abiertos['Fecha compromiso de cobertura'] == trasmañana)
tickets_abiertos,_ =ticket_abiertos_proceden.shape



# CUMPLIMIENTO DE RUTINAS

In [45]:
rutinas

,ID Cedula,ID Sitio People Center,Sitio,Turno,Día,Fecha Entrada,Hora Entrada,Fecha Salida,Hora Salida,Objetivo Rutinas,Rutinas Asignadas,Rutinas Realizadas,Geolocalización
0,89253.0,355,EUROVI ACOXPA,Nocturno,Martes,2025-07-22,01:42:00,2025-07-22,04:17:44,245,15,15,"19.2986496,-99.1368514"
1,89254.0,346,PLAY CITY ANTARA,Nocturno,Martes,2025-07-22,05:26:12,2025-07-22,05:32:52,65,16,16,"19.4393798,-99.2037186"
2,89255.0,191,MIYANA CENTRO COMERCIAL,Vespertino,Martes,2025-07-22,05:29:02,2025-07-22,05:32:56,142,56,56,"19.4402913,-99.1995188"
3,89270.0,301,SOFTTEK TOREO,Matutino,Martes,2025-07-22,06:09:18,2025-07-22,11:39:47,76,52,52,NaN
4,89271.0,93,SOFTTEK POLANCO,Matutino,Martes,2025-07-22,06:13:11,2025-07-22,10:56:49,31,10,10,"19.4379134,-99.2096785"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,NaN,0,PRUEBAS TECH2B,Matutino,Martes,2025-07-22,NaN,NaT,NaN,94,21,0,NaN
115,NaN,0,PRUEBAS TECH2B,Vespertino,Martes,2025-07-22,NaN,NaT,NaN,94,21,0,NaN
116,NaN,0,PRUEBAS TECH2B,Mixto,Martes,2025-07-22,NaN,NaT,NaN,94,52,0,NaN
117,NaN,93,SOFTTEK POLANCO,Mixto,Martes,2025-07-22,NaN,NaT,NaN,31,10,0,NaN


In [46]:

rutinas = rutinas.dropna(subset=['ID Cedula'])
total_rutinas =round(porcentaje(rutinas['Rutinas Asignadas'],rutinas['Rutinas Realizadas']),1)


In [47]:
print(f'{ticket_cerrados}')
print(f'{ticket_nuevos}')
print(f'{ticket_xvencer}')
print(f'{tickets_abiertos}')
print(f'{total_rutinas}')

7
2
1
30
98.9


# VACANTES

In [48]:
vacantes

,id,Fecha del reporte,Sitio,Plantilla,Activos,Vacante,Turno,Tipo,Ejecutivo
0,241,2025-03-01,LIVERPOOL PERINORTE,14,11,3,Matutino,Natural,Karina Sánchez
1,358,2025-03-01,AMERICAN EXPRESS POLANCO,14,15,0,Matutino,Natural,Ángel Cervantes
2,357,2025-03-01,LIVERPOOL JILOTEPEC,13,13,0,Matutino,Natural,Oscar Castillo
3,183,2025-03-01,LIVERPOOL INTERLOMAS,27,27,0,Matutino,Natural,Abel Vázquez
4,80,2025-03-01,CUMBRES PRIMARIA Y KINDER,7,8,0,Matutino,Natural,Ángel Cervantes
...,...,...,...,...,...,...,...,...,...
56787,101,2025-07-22,UNIVERSIDAD VIRTUAL DE LIVERPOOL,0,0,0,Mixto,Natural,NaN
56788,95,2025-07-22,TELEMUNDO,0,0,0,Mixto,Natural,NaN
56789,93,2025-07-22,SOFTTEK POLANCO,0,0,0,Mixto,Natural,NaN
56790,78,2025-07-22,PM STEELE,0,0,0,Mixto,Natural,NaN


In [49]:
# duplicados = vacantes.groupby(['Sitio', 'Turno', 'Fecha del reporte']).size()
# duplicados = duplicados[duplicados > 1]
# print(duplicados)

In [50]:

vacantes['Fecha del reporte'] = pd.to_datetime(vacantes['Fecha del reporte'])
vacantes['Fecha del reporte'] = vacantes['Fecha del reporte'].dt.date
vacantes = vacantes[['Sitio','Turno','Fecha del reporte','Vacante']]
tabla_pivot = vacantes.pivot(
    index=['Sitio', 'Turno'],
    columns='Fecha del reporte',
    values='Vacante', 
)

tabla_pivot = tabla_pivot.reset_index()
tabla_pivot = tabla_pivot.fillna(0)
fechas = tabla_pivot.columns[2:]

In [51]:
total_vacantes = np.sum(tabla_pivot.iloc[:,-1])
total_vacantes = int(total_vacantes)

In [52]:
vacantes_cubiertas = {}
for i in range(len(fechas) - 1):
    vacantes_cubiertas[f'cubiertas: {fechas[i]} - {fechas[i+1]}'] = (
        tabla_pivot[fechas[i]] - tabla_pivot[fechas[i+1]]
    ).clip(lower=0)

vacantes_nuevas = {}
for i in range(len(fechas) - 1):
    vacantes_nuevas[f'Nuevas: {fechas[i+1]} - {fechas[i]}'] = (
        tabla_pivot[fechas[i+1]] - tabla_pivot[fechas[i]]
    ).clip(lower=0)

sinmov = {}
for i in range(len(fechas) - 1):
    sinmov[f'Sin movimiento: min({fechas[i+1]},{fechas[i]})'] = tabla_pivot[[fechas[i+1], fechas[i]]].min(axis=1)

In [53]:
vacantes_cubiertas = pd.DataFrame(vacantes_cubiertas).fillna(0)
vacantes_cubiertas_totales = int(np.sum(np.array(vacantes_cubiertas.iloc[:,-1])))
vacantes_nuevas = pd.DataFrame(vacantes_nuevas).fillna(0)
vacantes_nuevas_totales = int(np.sum(np.array(vacantes_nuevas.iloc[:,-1])))
sinmov = pd.DataFrame(sinmov).fillna(0)
vacantes_sinmov_totales= int(np.sum(np.array(sinmov.iloc[:,-1])))

In [54]:
print(vacantes_cubiertas_totales)
print(vacantes_nuevas_totales)
print(vacantes_sinmov_totales)
print(total_vacantes)

10
9
30
39


In [55]:
def contar_no_ceros_consecutivos(fila):
    contador = 0
    for val in reversed(fila):
        if val != 0:
            contador += 1
        else:
            break
    return contador

conteos = sinmov.apply(contar_no_ceros_consecutivos, axis=1)


conteos= [x+1 if x!=0 else 0 for x in conteos ]


sinmov['Dias con vacantes']=conteos
sinmov['Turno'] = tabla_pivot['Turno']
sinmov['Sitios'] = tabla_pivot['Sitio']
resumen=sinmov[['Sitios','Turno', 'Dias con vacantes']].sort_values(by='Dias con vacantes', ascending= False)

vacantes_top5 = resumen.head(5)

C:\Users\centro.control\AppData\Local\Temp\ipykernel_1192\4116121981.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for val in reversed(fila):


In [56]:
html_vacantes = "\n".join(
    f"<li>{row['Sitios']} ({row['Turno']}): <strong>{row['Dias con vacantes']} días</strong></li>"
    for _, row in vacantes_top5.iterrows()
)


# UB

In [57]:

ub_bajos= ub[['sitio','turno', 'Unnamed: 14']]
ub_certificados = ub.groupby(['sitio', 'turno']).agg(
    total = ('Unnamed: 14', 'count'),
    Certificados= ('Unnamed: 14','sum')
)

ub_certificados["% de certificados"] = ((ub_certificados['Certificados']*100/ub_certificados['total'])).round(2)
ub_certificados = ub_certificados.reset_index()

ub_certificados=ub_certificados[(ub_certificados['sitio'] != 'CAPACITADORES BENNETTS') & (ub_certificados['sitio'] != 'CEDIS JILOTEPEC') &
(ub_certificados['sitio'] != 'TOKS CEDIS SINERGIA')  & (ub_certificados['sitio'] != 'PRE ALTAS')]


top_5_ub = ub_certificados[['sitio','turno',"% de certificados"]].sort_values(by="% de certificados").head(5)

In [58]:
top_5_ub

,sitio,turno,% de certificados
52,CL01601 DESPEGAR,Matutino,0.00
87,CL02901 KRISPY KREME,Nocturno,0.00
77,CL02501 GATX,Vespertino,0.00
68,CL02202 EUROVI LA ROSA,Vespertino,0.00
192,MAZDA CORPORATIVO,Matutino,16.67


In [59]:
html_ub = "\n".join(
    f"<li>{filas['sitio']} ({filas['turno']}): <strong>{filas['% de certificados']} %</strong></li>"
    for _, filas in top_5_ub.iterrows()
)

# Importar a HTML

In [60]:
# Leer tu archivo HTML base
with open("C:\\Users\\centro.control\\Documents\\code (1) - copia.html", "r", encoding="utf-8") as f:
    html = f.read()

# Reemplazar los marcadores por los valores
html = html.replace("{{ticket_abiertos}}", str(tickets_abiertos))
html = html.replace("{{ticket_nuevos}}", str(ticket_nuevos))
html = html.replace("{{ticket_cerrados}}", str(ticket_cerrados))
html = html.replace("{{ticket_xvencer}}", str(ticket_xvencer))
html = html.replace("{{%_de_Rutinas}}", str(total_rutinas)+"%")
html = html.replace("{{lista_vacantes}}", html_vacantes)
html = html.replace("{{lista_ub}}", html_ub)
html = html.replace("{{fecha}}", str(hoy))
html = html.replace("{{vacantes_cubiertas_totales}}", str(vacantes_cubiertas_totales))
html = html.replace("{{Total_de_vacantes_actuales}}", str(total_vacantes))
html = html.replace("{{vacantes_nuevas_total}}", str(vacantes_nuevas_totales))
html = html.replace("{{vacantes_sinmov_totales}}", str(vacantes_sinmov_totales))


# Guardar el HTML con los valores ya insertados
with open("C:\\Users\\centro.control\\Documents\\reporte_tickets.html", "w", encoding="utf-8") as f:
    f.write(html)